# Import all nessesary libary

In [1]:
import keras
import tensorflow as tf
import numpy as np
import os
import imageio as im
import cv2 

import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.training import saver as saver_lib
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from keras import backend as K

Using TensorFlow backend.


# load keras model

In [2]:
model = keras.models.load_model("inception.h5")
model.summary()

W0725 18:16:06.343153 139916207650112 deprecation_wrapper.py:119] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 18:16:06.387332 139916207650112 deprecation_wrapper.py:119] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 18:16:06.405599 139916207650112 deprecation_wrapper.py:119] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 18:16:06.406181 139916207650112 deprecation_wrapper.py:119] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0725 18:16:06.406755 1

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

/usr/local/lib64/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# load images

In [3]:
rootD = "images/"

X = []
for d, _, fileList in os.walk(rootD):
    for j, file in enumerate(fileList):
        img = im.imread(os.path.join(d, file))
        
        img = cv2.resize(img, (224, 224))
        img = np.array(img / 255)
        img = img[:,:,:3]
        X.append(img)

X = np.array(X)

# keras prediction time

In [4]:
%%time
%%timeit -r2 -n5
model.predict(X)

322 ms ± 88.5 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)
CPU times: user 9.51 s, sys: 221 ms, total: 9.73 s
Wall time: 3.22 s


# to tf model

In [4]:
# The export path contains the name and the version of the model
tf.keras.backend.set_learning_phase(0) # Ignore dropout at inference
frozen_model_path = './tf_model/1'

out_names = ",".join([layer.name.split(":")[0]  for layer in model.outputs])
inp_names = ",".join([layer.name.split(":")[0]  for layer in model.inputs])

print("OUTPUT: {}".format(out_names))
print("INPUT: {}".format(inp_names))
#model.summary()

K.set_learning_phase(0)
sess = K.get_session()
saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)
checkpoint_path = saver.save(sess, './saved_ckpt', global_step=0, latest_filename='checkpoint_state')
graph_io.write_graph(sess.graph, '.', './tmp.pb')
freeze_graph.freeze_graph('./tmp.pb', '',
                        False, checkpoint_path, out_names,
                        "save/restore_all",
                        "save/Const:0",
                        frozen_model_path,
                        False, "")

NameError: name 'model' is not defined

### load tf model

In [4]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ["serve"], "./tf_model/1/")
    graph = tf.get_default_graph()
    #print(graph.get_operations())
    
sess_config = tf.ConfigProto()
sess = tf.Session(graph=graph, config=sess_config)

W0725 18:11:50.614639 140700460901696 deprecation.py:323] From <ipython-input-4-9e9dc6d132a8>:2: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0725 18:11:51.443654 140700460901696 deprecation.py:323] From /usr/local/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


[<tf.Operation 'input_1' type=Placeholder>, <tf.Operation 'conv2d_1/kernel/Initializer/random_uniform/shape' type=Const>, <tf.Operation 'conv2d_1/kernel/Initializer/random_uniform/min' type=Const>, <tf.Operation 'conv2d_1/kernel/Initializer/random_uniform/max' type=Const>, <tf.Operation 'conv2d_1/kernel/Initializer/random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'conv2d_1/kernel/Initializer/random_uniform/sub' type=Sub>, <tf.Operation 'conv2d_1/kernel/Initializer/random_uniform/mul' type=Mul>, <tf.Operation 'conv2d_1/kernel/Initializer/random_uniform' type=Add>, <tf.Operation 'conv2d_1/kernel' type=VarHandleOp>, <tf.Operation 'conv2d_1/kernel/IsInitialized/VarIsInitializedOp' type=VarIsInitializedOp>, <tf.Operation 'conv2d_1/kernel/Assign' type=AssignVariableOp>, <tf.Operation 'conv2d_1/kernel/Read/ReadVariableOp' type=ReadVariableOp>, <tf.Operation 'conv2d_1/dilation_rate' type=Const>, <tf.Operation 'conv2d_1/Conv2D/ReadVariableOp' type=ReadVariableOp>, <tf.Operation 

### tf prediction

In [30]:
sess.run(["predictions/Softmax:0"], feed_dict={"input_1:0":X})

FailedPreconditionError: Error while reading resource variable batch_normalization_79/moving_variance from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/batch_normalization_79/moving_variance)
	 [[node batch_normalization_79/FusedBatchNorm/ReadVariableOp_1 (defined at <ipython-input-27-9e9dc6d132a8>:2) ]]

Original stack trace for 'batch_normalization_79/FusedBatchNorm/ReadVariableOp_1':
  File "/usr/lib64/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib64/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/ker

# Model optimization

### Post-training quantization